## Imports

In [1]:
from pathlib import Path
import re, os, csv, shutil
from collections import Counter, defaultdict
import pandas as pd
import hashlib
import zipfile

## Merging: Datasets 16 to 20 --> Workflow duplicated for every 5 datasets

In [2]:
BASE = Path("/kaggle/input")
DATASETS = [BASE/f"cdataset{i}" for i in range(16, 21)]
IMG_EXTS = {".jpg",".jpeg",".png",".bmp",".tif",".tiff",".webp"}

SKIP_CLASS_DIRS = {
    "_",
    "fleas_vs._healthy_dog_skin___v2_2023_09_17_1_50pm",
}

def normalize_class(name: str) -> str:
    s = name.strip().lower()
    s = s.replace("-", "_")
    s = re.sub(r"\s+", "_", s)
    s = s.replace("lesions", "lesion")
    s = s.replace("skinlesion", "skin_lesion").replace("skin__lesion","skin_lesion")
    s = s.replace("skin__infection","skin_infection")
    s = s.replace("healthy_dog_skin","healthy").replace("health_dog_skin","healthy").replace("healthydogskin","healthy")
    s = s.replace("no_disease","healthy").replace("normal_skin","healthy")
    s = s.replace("bacterial_dermatosis","bacterial_infection")
    s = s.replace("hypersensitivity_dermatitis","hypersensitivity")
    s = s.replace("fungal infection","fungal_infection")  # from cdataset10
    s = s.strip("_")
    return s

def is_image(p: Path) -> bool:
    return p.is_file() and p.suffix.lower() in IMG_EXTS

def split_class_from_path(root: Path, p: Path):
    rel = p.relative_to(root).parts
    if len(rel) >= 3 and rel[0] in {"train","valid","test"}:
        return rel[0], rel[1]
    if len(rel) >= 4 and rel[1] in {"train","valid","test"}:
        return rel[1], rel[2]
    return None, None

manifest = []          
skipped  = [] 

for root in DATASETS:
    if not root.exists(): 
        skipped.append((root.name, "dataset_missing", "", "", str(root)))
        continue
    for p in root.rglob("*"):
        if not is_image(p): 
            continue
        split, raw_cls = split_class_from_path(root, p)
        if split is None:
            skipped.append((root.name, "layout_unrecognized", "", "", str(p)))
            continue
        if raw_cls in SKIP_CLASS_DIRS:
            skipped.append((root.name, "bad_class", split, raw_cls, str(p)))
            continue
        norm_cls = normalize_class(raw_cls)
        manifest.append((root.name, split, raw_cls, norm_cls, str(p)))

print("Kept rows:", len(manifest))
print("Skipped rows:", len(skipped))

# saving skipped list
WORK = Path("/kaggle/working")
WORK.mkdir(exist_ok=True)
skip_csv = WORK/"skipped_cdataset16_20.csv"
with open(skip_csv, "w", newline="", encoding="utf-8") as f:
    w = csv.writer(f)
    w.writerow(["dataset","reason","split","raw_class","path"])
    w.writerows(skipped)

Kept rows: 7827
Skipped rows: 0


In [3]:
df = pd.DataFrame(manifest, columns=["dataset","split","raw_class","class","path"])

print("Totals by split:")
print(df.groupby("split").size())

print("\nTop raw class names:")
for k,v in Counter(df["raw_class"]).most_common(20):
    print(f"  {k}: {v}")

print("\nTop normalized class names:")
for k,v in Counter(df["class"]).most_common(20):
    print(f"  {k}: {v}")

Totals by split:
split
test      584
train    6020
valid    1223
dtype: int64

Top raw class names:
  ringworm: 1068
  healthy: 902
  demodicosis: 858
  bacterial dermatosis: 854
  hypersensitivity dermatitis: 729
  Dermatitis: 640
  Healthy: 469
  Fungal_infections: 429
  fungal infection: 343
  BACTERIAL_INFECTION: 317
  FUNGAL_INFECTION: 315
  Hypersensitivity: 236
  fleas: 225
  HEALTHY: 174
  Bacterial_dermatosis: 78
  flea_allergy: 75
  mange: 62
  hotspot: 33
  Hypersensitivity_allergic_dermatosis: 20

Top normalized class names:
  healthy: 1545
  bacterial_infection: 1249
  ringworm: 1068
  hypersensitivity: 965
  demodicosis: 858
  fungal_infection: 658
  dermatitis: 640
  fungal_infections: 429
  fleas: 225
  flea_allergy: 75
  mange: 62
  hotspot: 33
  hypersensitivity_allergic_dermatosis: 20


In [4]:
MERGED = Path("/kaggle/working/merged16_20")
USE_SYMLINKS = True 

# checking that merged dataset exists
if MERGED.exists():
    shutil.rmtree(MERGED)
for sp in ["train","valid","test"]:
    (MERGED/sp).mkdir(parents=True, exist_ok=True)

moved = 0
for sp, cls, src in df[["split","class","path"]].itertuples(index=False):
    src_p = Path(src)
    dst_dir = MERGED/sp/cls
    dst_dir.mkdir(parents=True, exist_ok=True)
    dst = dst_dir/src_p.name
    try:
        if USE_SYMLINKS:
            if not dst.exists(): os.symlink(src_p, dst)
        else:
            if not dst.exists(): shutil.copy2(src_p, dst)
        moved += 1
    except FileExistsError:
        pass

print(f"Staged {moved} files into {MERGED}")

Staged 7827 files into /kaggle/working/merged16_20


In [5]:
IMG_EXTS = {".jpg",".jpeg",".png",".bmp",".tif",".tiff",".webp"}

def count_merged(root: Path):
    rows = []
    for sp in ["train","valid","test"]:
        sp_dir = root/sp
        if not sp_dir.exists(): 
            continue
        for cls_dir in sp_dir.iterdir():
            if cls_dir.is_dir():
                n = sum(1 for p in cls_dir.rglob("*") if p.is_file() and p.suffix.lower() in IMG_EXTS)
                rows.append((sp, cls_dir.name, n))
    return rows

rows = count_merged(Path("/kaggle/working/merged16_20"))
tab = pd.DataFrame(rows, columns=["split","class","count"]).sort_values(["split","class"])
display(tab)
print("\nTotals by split:")
print(tab.groupby("split")["count"].sum())

split                                 class  count
33   test                   bacterial_infection     59
32   test                           demodicosis    100
26   test                            dermatitis     46
30   test                          flea_allergy     19
27   test                                 fleas      9
31   test                      fungal_infection     32
29   test                     fungal_infections     37
23   test                               healthy    122
25   test                      hypersensitivity     37
24   test                                 mange     19
28   test                              ringworm     91
12  train                   bacterial_infection    979
10  train                           demodicosis    584
4   train                            dermatitis    469
8   train                          flea_allergy     56
5   train                                 fleas    132
9   train                      fungal_infection    554
7   train                     fungal_infections    335
0   train                               healthy   1150
11  train                               hotspot     33
3   train                      hypersensitivity    755
2   train  hypersensitivity_allergic_dermatosis     17
1   train                                 mange     43
6   train                              ringworm    794
22  valid                   bacterial_infection    211
21  valid                           demodicosis    174
16  valid                            dermatitis    125
17  valid                                 fleas      6
20  valid                      fungal_infection     72
19  valid                     fungal_infections     57
13  valid                               healthy    206
15  valid                      hypersensitivity    173
14  valid  hypersensitivity_allergic_dermatosis      3
18  valid                              ringworm    183


Totals by split:
split
test      571
train    5901
valid    1210
Name: count, dtype: int64


In [6]:
def file_hash(p, blocksize=65536):
    h = hashlib.md5()
    with open(p,'rb') as f:
        for chunk in iter(lambda: f.read(blocksize), b""):
            h.update(chunk)
    return h.hexdigest()

hash_map = defaultdict(list)
for p in MERGED.rglob("*"):
    if p.is_file() and p.suffix.lower() in IMG_EXTS:
        h = file_hash(p)
        hash_map[h].append(p)

dupes = {h:files for h,files in hash_map.items() if len(files) > 1}

print(f"Found {len(dupes)} duplicate groups")
total_dupes = sum(len(v)-1 for v in dupes.values())
print(f"Total duplicate files (beyond first copy): {total_dupes}")

removed = 0
for files in dupes.values():
    for f in files[1:]:
        f.unlink()
        removed += 1
print("Removed", removed, "duplicate files")

Found 0 duplicate groups
Total duplicate files (beyond first copy): 0
Removed 0 duplicate files


In [7]:
rows = []
for sp in ["train","valid","test"]:
    for cls_dir in (MERGED/sp).iterdir():
        if cls_dir.is_dir():
            n = sum(1 for p in cls_dir.rglob("*") if p.is_file() and p.suffix.lower() in IMG_EXTS)
            rows.append((sp, cls_dir.name, n))

tab = pd.DataFrame(rows, columns=["split","class","count"]).sort_values(["split","class"])
display(tab)
print("\nTotals by split:")
print(tab.groupby("split")["count"].sum())

split                                 class  count
33   test                   bacterial_infection     59
32   test                           demodicosis    100
26   test                            dermatitis     46
30   test                          flea_allergy     19
27   test                                 fleas      9
31   test                      fungal_infection     32
29   test                     fungal_infections     37
23   test                               healthy    122
25   test                      hypersensitivity     37
24   test                                 mange     19
28   test                              ringworm     91
12  train                   bacterial_infection    979
10  train                           demodicosis    584
4   train                            dermatitis    469
8   train                          flea_allergy     56
5   train                                 fleas    132
9   train                      fungal_infection    554
7   train                     fungal_infections    335
0   train                               healthy   1150
11  train                               hotspot     33
3   train                      hypersensitivity    755
2   train  hypersensitivity_allergic_dermatosis     17
1   train                                 mange     43
6   train                              ringworm    794
22  valid                   bacterial_infection    211
21  valid                           demodicosis    174
16  valid                            dermatitis    125
17  valid                                 fleas      6
20  valid                      fungal_infection     72
19  valid                     fungal_infections     57
13  valid                               healthy    206
15  valid                      hypersensitivity    173
14  valid  hypersensitivity_allergic_dermatosis      3
18  valid                              ringworm    183


Totals by split:
split
test      571
train    5901
valid    1210
Name: count, dtype: int64


In [8]:
# picking the source folder
candidates = [Path("/kaggle/working/merged16_20"), Path("/kaggle/working/merged16_20")]
src = next((p for p in candidates if p.exists()), None)
assert src is not None, "Couldn't find folder"

# copying to a staging folder to deference symlinks
stage = Path("/kaggle/working/_zip_stage_16_20")
if stage.exists():
    shutil.rmtree(stage)
shutil.copytree(src, stage, symlinks=False)  # follow links → copy actual bytes

# creating the zip in /kaggle/working
zip_base = "/kaggle/working/merged16_20"
zip_path = shutil.make_archive(zip_base, "zip", root_dir=stage)

# cleaning the staging folder to save space
shutil.rmtree(stage)

## Merging: All Datasets

In [9]:
# defining all dataset paths and wrapper dataset
DATASETS = [
    Path("/kaggle/input/cdatasets1-5"),
    Path("/kaggle/input/cdatasets6-10"),
    Path("/kaggle/input/cdatasets11-15"),
    Path("/kaggle/input/cdatasets16-20"),
    Path("/kaggle/input/cdatset21"),  
]
IMG_EXTS = {".jpg",".jpeg",".png",".bmp",".tif",".tiff",".webp"}

# normalization to unify known variants
def normalize_class(name: str) -> str:
    s = name.strip().lower().replace("-", "_")
    s = re.sub(r"\s+", "_", s)
    # canonical names
    s = (s.replace("bacterial dermatosis","bacterial_infection")
           .replace("bacterial_dermatosis","bacterial_infection"))
    s = (s.replace("hypersensitivity dermatitis","hypersensitivity")
           .replace("hypersensitivity_dermatitis","hypersensitivity")
           .replace("hypersensitivity_allergic_dermatosis","hypersensitivity"))
    s = (s.replace("fungal infection","fungal_infection")
           .replace("fungal_infections","fungal_infection"))
    s = (s.replace("healthy_dog_skin","healthy")
           .replace("health_dog_skin","healthy")
           .replace("healthydogskin","healthy")
           .replace("no_disease","healthy")
           .replace("normal_skin","healthy"))
    s = (s.replace("lesions","lesion")
           .replace("skin__lesion","skin_lesion")
           .replace("skinlesion","skin_lesion"))
    # your call: fold ambiguous 'ill' into a symptom bucket
    s = s.replace("ill","skin_lesion")
    return s.strip("_")

def is_image(p: Path): 
    return p.is_file() and p.suffix.lower() in IMG_EXTS

ALIAS_TO_CANON = {a:k for k,vs in {
    "train":{"train","training","train_set","trainset"},
    "valid":{"valid","validation","val","valid_set","val_set"},
    "test":{"test","testing","test_set","testset"},
}.items() for a in vs}

def find_split_and_class(root: Path, p: Path):
    parts = list(p.relative_to(root).parts)
    for i in range(max(0,len(parts)-6), len(parts)-1):
        token = parts[i].lower()
        split = ALIAS_TO_CANON.get(token, token if token in {"train","valid","test"} else None)
        if split and i+1 < len(parts)-1:
            return split, parts[i+1]
    return None, None

manifest, skipped = [], []
for root in DATASETS:
    if not root.exists():
        skipped.append((root, "dataset_missing"))
        continue
    for p in root.rglob("*"):
        if not is_image(p): 
            continue
        split, raw_cls = find_split_and_class(root, p)
        if split is None:
            skipped.append((p, "layout_unrecognized"))
            continue
        manifest.append((root.name, split, raw_cls, normalize_class(raw_cls), str(p)))

print("Kept rows (all roots):", len(manifest))
print("Skipped (layout/missing):", len(skipped))

df_all = pd.DataFrame(manifest, columns=["dataset","split","raw_class","class","path"])

print("\nTotals by split (pre-merge):")
print(df_all.groupby("split").size())
print("\nTop classes (normalized, pre-merge):")
print(df_all["class"].value_counts().head(20))

Kept rows (all roots): 36810
Skipped (layout/missing): 0

Totals by split (pre-merge):
split
test      3173
train    28592
valid     5045
dtype: int64

Top classes (normalized, pre-merge):
class
skin_lesion            6756
healthy                5994
ringworm               3688
fungal_infection       3248
bacterial_infection    2810
demodicosis            2789
hypersensitivity       2655
eye_redness            2513
nasal_discharge        1818
keratosis              1733
dermatitis             1270
hotspot                 486
flea_allergy            457
mange                   368
fleas                   225
Name: count, dtype: int64


In [10]:
FINAL = Path("/kaggle/working/final_1_21_fixed")
IMG_EXTS = {".jpg",".jpeg",".png",".bmp",".tif",".tiff",".webp"}

PRIORITY = {"test": 0, "valid": 1, "train": 2}
if FINAL.exists():
    shutil.rmtree(FINAL)
for sp in ["train","valid","test"]:
    (FINAL/sp).mkdir(parents=True, exist_ok=True)

# keep-first on filename collisions
collisions = []
moved = 0
for ds, sp, cls, src in df_all[["dataset","split","class","path"]].itertuples(index=False):
    src_p = Path(src)
    out_dir = FINAL/sp/cls
    out_dir.mkdir(parents=True, exist_ok=True)
    dst = out_dir/src_p.name
    if dst.exists():
        collisions.append((sp, cls, src_p.name, str(src_p), str(dst)))
        continue
    os.symlink(src_p, dst)
    moved += 1

print(f"Staged {moved} files -> {FINAL}")
with open("/kaggle/working/collisions_final_1_21.csv","w",newline="",encoding="utf-8") as f:
    w = csv.writer(f); w.writerow(["split","class","filename","source_path","kept_path"]); w.writerows(collisions)

# count per split × class after staging
rows=[]
for sp in ["train","valid","test"]:
    sp_dir = FINAL/sp
    if not sp_dir.exists(): continue
    for cls_dir in sp_dir.iterdir():
        if cls_dir.is_dir():
            n = sum(1 for q in cls_dir.rglob("*") if q.is_file() and q.suffix.lower() in IMG_EXTS)
            rows.append((sp, cls_dir.name, n))
tab_stage = pd.DataFrame(rows, columns=["split","class","count"]).sort_values(["split","class"])
display(tab_stage)
print("\nTotals by split (post-merge, pre-dedup):")
print(tab_stage.groupby("split")["count"].sum())

Staged 36767 files -> /kaggle/working/final_1_21_fixed


split                class  count
44   test  bacterial_infection    133
42   test          demodicosis    308
34   test           dermatitis     93
36   test          eye_redness    352
40   test         flea_allergy     25
35   test                fleas     18
41   test     fungal_infection    236
30   test              healthy    343
43   test              hotspot     12
32   test     hypersensitivity    131
33   test            keratosis    195
31   test                mange     28
39   test      nasal_discharge    194
37   test             ringworm    303
38   test          skin_lesion    798
14  train  bacterial_infection   2361
12  train          demodicosis   2026
4   train           dermatitis    915
6   train          eye_redness   1716
10  train         flea_allergy    422
5   train                fleas    195
11  train     fungal_infection   2692
0   train              healthy   5100
13  train              hotspot    446
2   train     hypersensitivity   2218
3   train            keratosis   1155
1   train                mange    328
9   train      nasal_discharge   1221
7   train             ringworm   2884
8   train          skin_lesion   4883
29  valid  bacterial_infection    316
27  valid          demodicosis    455
19  valid           dermatitis    262
21  valid          eye_redness    439
25  valid         flea_allergy     10
20  valid                fleas     12
26  valid     fungal_infection    320
15  valid              healthy    551
28  valid              hotspot     28
17  valid     hypersensitivity    306
18  valid            keratosis    383
16  valid                mange     12
24  valid      nasal_discharge    403
22  valid             ringworm    501
23  valid          skin_lesion   1038


Totals by split (post-merge, pre-dedup):
split
test      3169
train    28562
valid     5036
Name: count, dtype: int64


In [11]:
# final merged folder
FINAL = Path("/kaggle/working/final_1_21")        
IMG_EXTS = {".jpg",".jpeg",".png",".bmp",".tif",".tiff",".webp"}
PRIORITY = {"test": 0, "valid": 1, "train": 2}   

def file_hash(p: Path, bs=65536):
    h = hashlib.md5()
    with open(p,'rb') as f:
        for chunk in iter(lambda: f.read(bs), b""):
            h.update(chunk)
    return h.hexdigest()

# building a proper file index
index_rows = []
for sp in ["test","valid","train"]:
    sp_dir = FINAL/sp
    if not sp_dir.exists(): 
        continue
    for cls_dir in sp_dir.iterdir():
        if not cls_dir.is_dir():
            continue
        for p in cls_dir.iterdir():
            if p.is_file() and p.suffix.lower() in IMG_EXTS:
                try:
                    h = file_hash(p)
                    index_rows.append((h, sp, cls_dir.name, p))
                except Exception:
                    pass

# grouping by content hash and keep highest-priority copy
by_hash = defaultdict(list)
for h, sp, cls, p in index_rows:
    by_hash[h].append((sp, cls, p))

conflicts, removed = [], 0
for h, items in by_hash.items():
    items_sorted = sorted(items, key=lambda x: (PRIORITY.get(x[0], 99), x[1]))
    keep_sp, keep_cls, keep_path = items_sorted[0]

    # log label conflicts
    classes = {c for s, c, _ in items}
    if len(classes) > 1:
        conflicts.append((h, keep_sp, keep_cls, str(keep_path), sorted(classes)))

    # delete all non-kept copies
    for s, c, p in items_sorted[1:]:
        try:
            p.unlink()
            removed += 1
        except FileNotFoundError:
            pass

print("Removed duplicates with split priority:", removed)
print("Label-conflict groups:", len(conflicts))

with open("/kaggle/working/label_conflicts.csv","w",newline="",encoding="utf-8") as f:
    w = csv.writer(f)
    w.writerow(["hash","kept_split","kept_class","kept_path","all_classes"])
    w.writerows(conflicts)

# recount per split × class 
counts = []
for sp in ["test","valid","train"]:
    sp_dir = FINAL/sp
    if not sp_dir.exists(): 
        continue
    for cls_dir in sp_dir.iterdir():
        if cls_dir.is_dir():
            n = sum(1 for q in cls_dir.rglob("*") if q.is_file() and q.suffix.lower() in IMG_EXTS)
            counts.append((sp, cls_dir.name, n))

tab_final = pd.DataFrame(counts, columns=["split","class","count"]).sort_values(["split","class"])
display(tab_final)
print("\nTotals by split:")
print(tab_final.groupby("split")["count"].sum())
print("Grand total:", tab_final["count"].sum())

Removed duplicates with split priority: 0
Label-conflict groups: 0


Empty DataFrame
Columns: [split, class, count]
Index: []


Totals by split:
Series([], Name: count, dtype: object)
Grand total: 0


In [20]:
FINAL = Path("/kaggle/working/collisions_final_1_21.csv")
ZIP_PATH = "/kaggle/working/final_1_21_clean.zip"

with zipfile.ZipFile(ZIP_PATH, "w", zipfile.ZIP_DEFLATED) as zipf:
    zipf.write(FINAL, arcname=FINAL.name)

print("Created ZIP Successfully!")


Created ZIP Successfully!
